# **PostgreSQL ile performans testi deneyimi**

<div style="text-align:center;"><img src="img/postgre_exp_erd.png" /></div>

<h2>PostgreSQL bağlantısı</h2>

<b>psycopg2</b> kütüphanesiyle PostgreSQL veritabanına bağlantı kurulur. DB parametrelerini config dosyasından almak için <b>pyyaml</b> kütüphanesi kullanılır.

Öncesinde DB'ye manuel bağlanıp (pgAdmin vb.) bir veritabanı oluşturulması gerekir (bu örnekte <b>perf_test_db</b>).

In [1]:
# Imports

import psycopg2
from psycopg2 import Error
import yaml
from datetime import datetime

In [2]:
# Get parameters from config file

with open("config.yml", "r") as ymlfile:
    cfg = yaml.safe_load(ymlfile)

p_dbname = cfg["postgre"]["dbname"]
p_port = cfg["postgre"]["port"]
p_user= cfg["postgre"]["user"]
p_host = cfg["postgre"]["host"]
p_password = cfg["postgre"]["password"]

In [3]:
try:

    # Connect to an existing database
    connection = psycopg2.connect(
                                    dbname = p_dbname,
                                    port = p_port,
                                    user= p_user,
                                    host = p_host,
                                    password = p_password
                                )

    # Create a cursor to perform database operations
    cursor = connection.cursor()

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

<h2>Index ile performans testi</h2>

<h3>1 - Normal sorgu</h3>

Herhangi bir performans iyileştirmesi (index, partition, vs.) uygulanmadan sorgu süreleri test edilir.

In [4]:
ilk = datetime.now()

sorgu_index_normal = """
                    select c.FirstName, c.UserName, p.ProductID, count(o.OrderID) as total
                    from orders o
                    left join products p
                        on o.ProductID = p.ProductID
                    left join Customers c
                        on o.CustomerID = c.CustomerID
                    where o.OrderDate > '2022-06-01'
                        and c.UserName like '%gmail%'
                    group by c.FirstName, c.UserName, p.ProductID
"""

cursor.execute(sorgu_index_normal)

son = datetime.now()

sure_index_normal = son - ilk

print("Normal çalışma süresi: ", sure_index_normal)

Normal çalışma süresi:  0:04:12.545279


<h3> 2 - OrderDate üzerinde index oluşturulmuş sorgu</h3>

**Orders** tablosunun bir kopyası oluşturulur (Orders_Index):

<br />

``` sql
drop table if exists Customers_Index;
create table Customers_Index as select * from customers;

drop table if exists Orders_Index;
create table Orders_Index as select * from orders;
```

<br />

Orders_Index tablosu üzerinde **OrderDate** kolonu üzerinde index tanımlanır:

<br />

``` sql
create index idx_order_orderdate on Orders_Index (OrderDate);
drop index if exists idx_order_orderdate;
```

<br />

Customers_Index tablosu üzerinde **UserName** kolonu üzerinde index tanımlanır:

<br />

``` sql
create index idx_customer_username on Customers_Index (UserName);
drop index if exists idx_customer_username;
```

<br />

In [5]:
ilk = datetime.now()

sorgu_index_hizli = """
                        select c.FirstName, c.UserName, p.ProductID, count(o.OrderID) as total
                        from Orders_Index o
                        left join products p
                            on o.ProductID = p.ProductID
                        left join Customers_Index c
                            on o.CustomerID = c.CustomerID
                        where o.OrderDate = '2022-06-22'
                            and c.UserName like '%gmail%'
                        group by c.FirstName, c.UserName, p.ProductID
"""

cursor.execute(sorgu_index_hizli)

son = datetime.now()

sure_index_hizli = son - ilk

print("Index ile çalışma süresi: ", sure_index_hizli)

Index ile çalışma süresi:  0:00:19.563190


<h2>Partition ile performans testi</h2>

<h3>1 - Normal Sorgu</h3>

Herhangi bir performans iyileştirmesi (index, partition, vs.) uygulanmadan sorgu süreleri test edilir.

In [6]:
ilk = datetime.now()

sorgu_partition_normal = """
                    select p.Category, count(o.OrderID) as total
                    from orders o
                    left join products p
                        on o.ProductID = p.ProductID
                    where o.OrderDate = '2022-06-22'
                    group by p.Category
"""

cursor.execute(sorgu_partition_normal)

son = datetime.now()

sure_partition_normal = son - ilk

print("Normal çalışma süresi: ", sure_partition_normal)

Normal çalışma süresi:  0:00:11.593361


<h3>2 - OrderDate üzerinde aya göre partition oluşturulmuş sorgu</h3>

**Orders** tablosunun bir kopyası oluşturulur (Orders_Partition):

<br />

``` sql
create table Orders_Partition
(
  OrderID numeric(10),
  ProductID numeric(10),
  CustomerID numeric(10),
  Quantity int,
  OrderDate date
) partition by range (OrderDate)
;
```
<br />

Bu tablo üzerinde **OrderDate** kolonu üzerinde aya göre, 12 tane partition tanımlanır:

<br />

``` sql
create table order_part_2022_1 partition OF orders_partition for values from ('2022-01-01') to ('2022-02-01');
create table order_part_2022_2 partition OF orders_partition for values from ('2022-02-01') to ('2022-03-01');
create table order_part_2022_3 partition OF orders_partition for values from ('2022-03-01') to ('2022-04-01');
create table order_part_2022_4 partition OF orders_partition for values from ('2022-04-01') to ('2022-05-01');
create table order_part_2022_5 partition OF orders_partition for values from ('2022-05-01') to ('2022-06-01');
create table order_part_2022_6 partition OF orders_partition for values from ('2022-06-01') to ('2022-07-01');
create table order_part_2022_7 partition OF orders_partition for values from ('2022-07-01') to ('2022-08-01');
create table order_part_2022_8 partition OF orders_partition for values from ('2022-08-01') to ('2022-09-01');
create table order_part_2022_9 partition OF orders_partition for values from ('2022-09-01') to ('2022-10-01');
create table order_part_2022_10 partition OF orders_partition for values from ('2022-10-01') to ('2022-11-01');
create table order_part_2022_11 partition OF orders_partition for values from ('2022-11-01') to ('2022-12-01');
create table order_part_2022_12 partition OF orders_partition for values from ('2022-12-01') to ('2023-01-01');
```

<br />

Daha sonra ilgili aylar her partition'a insert edilir:

<br />

``` sql
insert into Orders_Partition select * from orders where OrderDate >= '2022-01-01' and OrderDate < '2022-02-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-02-01' and OrderDate < '2022-03-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-03-01' and OrderDate < '2022-04-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-04-01' and OrderDate < '2022-05-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-05-01' and OrderDate < '2022-06-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-06-01' and OrderDate < '2022-07-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-07-01' and OrderDate < '2022-08-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-08-01' and OrderDate < '2022-09-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-09-01' and OrderDate < '2022-10-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-010-01' and OrderDate < '2022-11-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-011-01' and OrderDate < '2022-12-01';
insert into Orders_Partition select * from orders where OrderDate >= '2022-012-01' and OrderDate < '2023-01-01';
```

<br />

In [7]:
ilk = datetime.now()

sorgu_partition_hizli = """
                    select p.Category, count(o.OrderID) as total
                    from Orders_Partition o
                    left join products p
                        on o.ProductID = p.ProductID
                    where o.OrderDate = '2022-06-22'
                    group by p.Category
"""

cursor.execute(sorgu_partition_hizli)

son = datetime.now()

sure_partition_hizli = son - ilk

print("Partition ile çalışma süresi: ", sure_partition_hizli)

Partition ile çalışma süresi:  0:00:00.270212


<h3>Karşılaştırma</h3>

In [8]:
print(" Index Test", "\n", "-"*20)
print("Normal\t\t", sure_index_normal)
print("Index\t\t", sure_index_hizli)
print("Hız artışı (%)\t", ((sure_index_normal - sure_index_hizli) / sure_index_normal) * 100)
print("\n")
print(" Partition Test", "\n", "-"*20)
print("Normal\t\t", sure_partition_normal)
print("Partition\t", sure_partition_hizli)
print("Hız artışı (%)\t", ((sure_partition_normal - sure_partition_hizli) / sure_partition_normal) * 100)

 Index Test 
 --------------------
Normal		 0:04:12.545279
Index		 0:00:19.563190
Hız artışı (%)	 92.25359108771937


 Partition Test 
 --------------------
Normal		 0:00:11.593361
Partition	 0:00:00.270212
Hız artışı (%)	 97.66925225566598
